In the forward pass, the order is to compute $f(x)$ for the given mini-batch $x$ by sequentially applying the partitions $f^j$ as $x^j = f^j(x^{j-1})$ for $j = 1, \cdots, n$. This is done for each micro-batch $x_i$ using the tasks $F_{i, j}$, where $x_i^0 = x_i$ and $x_i^j \leftarrow f^j(x_i^{j-1})$ for $i = 1, \cdots, m$ and $j = 1, \cdots, n$. The forward pass proceeds in the order of increasing partition index $j$.

The backward pass, on the other hand, computes the gradient of the loss with respect to the network parameters $\theta$. It starts with the gradient of the loss with respect to the output $x_i^n$ and proceeds with the tasks $B_{i, j}$, where $d x_i^{j-1} \leftarrow \partial_x f^j(d x_i^j)$ and $g_i^j \leftarrow \partial_{\theta^j} f^j(d x_i^j)$ for $i = 1, \cdots, m$ and $j = 1, \cdots, n$. The backward pass goes in the order of decreasing partition index $j$.

The order of the backward pass needs to be modified due to the dependencies between tasks. Specifically, $B_{i, j}$ depends on the outcome of $B_{i, j+1}$, and this dependency requires that $B_{i, j+1}$ be completed before starting $B_{i, j}$. To efficiently parallelize the tasks, pipeline parallelism assigns tasks with respect to the partition index $j$ so that the $j$-th partition entirely lies in the $j$-th device, and enforces that $F_{i, j}$ must be completed before executing $F_{i+1, j}$ and $B_{i, j}$ must be completed before executing $B_{i-1, j}$.

Here's a concrete simple example from the passage: let's consider the case where we have 4 micro-batches ($m=4$) and 3 partitions ($n=3$). In the forward pass, we first compute $x_1^1 = f^1(x_1^0)$, then $x_1^2 = f^2(x_1^1)$, and finally $x_1^3 = f^3(x_1^2)$. Next, we compute the forward pass for the remaining micro-batches $x_2, x_3,$ and $x_4$. In the backward pass, we first compute $d x_1^2 \leftarrow \partial_x f^3(d x_1^3)$ and $g_1^3 \leftarrow \partial_{\theta^3} f^3(d x_1^3)$. Then, we compute $d x_1^1 \leftarrow \partial_x f^2(d x_1^2)$ and $g_1^2 \leftarrow \partial_{\theta^2} f^2(d x_1^2)$, followed by $d x_1^0 \leftarrow \partial_x f^1(d x_1^1)$ and $g_1^1 \leftarrow \partial_{\theta^1} f^1(d x_1^1)$. This process is repeated for the remaining micro-b




